# Exploration of `ExPeCoN`

Running analyses for `ExPeCoN`

results are published in Forster et al., 2024 (hopefully)
___

    Author:  Carina Forster et al.
    Contact: forster@cbs.mpg.de
    Years:   2023

___

Make sure you are in the right environment: expecon_3.9
We use nbstripout for seamless [version control](https://towardsdatascience.com/enhancing-data-science-workflows-mastering-version-control-for-jupyter-notebooks-b03c839e25ec) of jupyter notebooks:

nbstripout integrates with Git hooks to automatically strip output cells fromnotebooks when they are committed. 
It modifies the notebook’s JSON content,removing the output fields, thus reducing the file size and simplifying diffs.

## Setup 

In [ ]:
# turn off warnings for a cleaner output
import warnings

warnings.filterwarnings('ignore')

In [ ]:
%matplotlib inline  

In [ ]:
# import packages
import mne
import pandas as pd
import numpy as np
from pathlib import Path
import random
import matplotlib.pyplot as plt

# expecon_ms functions
from expecon_ms.configs import config, path_to

# import functions for behavioral analysis
from expecon_ms.behav import figure1 as behav

# Import functions from expecon_package for preproccesing eeg data
from expecon_ms.eeg.preprocessing import prepro as pp
from expecon_ms.eeg.preprocessing import ica

# import functions for EEG analysis and visualization
from expecon_ms.eeg.sensor import evokeds as evo
from expecon_ms.eeg.sensor import tfr_contrasts as tfr
from expecon_ms.eeg.sensor import tfr_trial

# import functions for source analysis
from expecon_ms.eeg.source import source_reco

In [ ]:
# Define the output for mne functions
import mne

mne.set_log_level('CRITICAL')

In [ ]:
# Set global vars & paths (unused at the moment)

# this doesn't work for some reason
# the variables set here are not recognized in the functions

# raw concatenated eeg data
save_dir_concatenated_raw1 = Path(path_to.data.eeg.RAW_expecon1)
save_dir_concatenated_raw2 = Path(path_to.data.eeg.RAW_expecon2)
save_dir_concatenated_raw1.mkdir(parents=True, exist_ok=True)
save_dir_concatenated_raw2.mkdir(parents=True, exist_ok=True)

# stimulus locked
save_dir_stim_1 = Path(path_to.data.eeg.preprocessed.stimulus_expecon1)
save_dir_stim_2 = Path(path_to.data.eeg.preprocessed.stimulus_expecon2)
save_dir_stim_1.mkdir(parents=True, exist_ok=True)
save_dir_stim_2.mkdir(parents=True, exist_ok=True)

# cue locked
save_dir_cue_1 = Path(path_to.data.eeg.preprocessed.cue_expecon1)
save_dir_cue_2 = Path(path_to.data.eeg.preprocessed.cue_expecon2)
save_dir_cue_1.mkdir(parents=True, exist_ok=True)
save_dir_cue_2.mkdir(parents=True, exist_ok=True)

# directory that contains the cleaned epochs
dir_clean_epochs_expecon1 = Path(path_to.data.eeg.preprocessed.ica.clean_epochs_expecon1)
dir_clean_epochs_expecon2 = Path(path_to.data.eeg.preprocessed.ica.clean_epochs_expecon2)

# EEG cap layout file
filename_montage = Path(path_to.data.templates)
filename_montage.mkdir(parents=True, exist_ok=True)

# raw behavioral data
behav_path = Path(path_to.data.behavior)
behav_path.mkdir(parents=True, exist_ok=True)

# participant IDs
id_list_expecon1 = config.participants.ID_list_expecon1
id_list_expecon2 = config.participants.ID_list_expecon2

# pilot data counter (for expecon 1, participant ID starts with ID007)
pilot_counter = config.participants.pilot_counter

# data_cleaning parameters defined in config.toml
rt_max = config.behavioral_cleaning.rt_max
rt_min = config.behavioral_cleaning.rt_min
hitrate_max = config.behavioral_cleaning.hitrate_max
hitrate_min = config.behavioral_cleaning.hitrate_min
farate_max = config.behavioral_cleaning.farate_max
hit_fa_diff = config.behavioral_cleaning.hit_fa_diff

## Analyse 

1. Behavioral data analysis (Signal detection theory based)

In [ ]:
# check the function arguments the docs
help(behav.plot_figure1_grid)

In [ ]:
behav.plot_figure1_grid(expecon=1, exclude_high_fa=True)

2. Preprocessing EEG data

In [ ]:
# function expects a raw object with .fif file ending
pp.prepro(study=2, trigger='stimulus', l_freq=1, h_freq=40, tmin=-1,
        tmax=1, resample_rate=250,
        sf=2500, detrend=1, ransac=1, autoreject=0)

# how many channels were interpolated?
pp.n_channels_interpolated(study=2, trigger='stimulus', l_freq=0.1)

# run ica on clean, epoched data
ica.run_ica(study=2, infomax=1, save_psd=1)
# correlate with EOG and ECG and mark bad componets for rejection

ica.label_ica_correlation(study=2)

# usa icalabel to mark components for rejection
#ica.label_iclabel(study=1)

ICA stats

In [ ]:
# which study to run the analysis on
study = 2

In [ ]:
# load the csv file that contains the number of components rejected
df = pd.read_csv(f"E:\expecon_ms\data\eeg\prepro_ica\clean_epochs_corr{str(study)}\ica_components_stats_icacorr.csv")

# mean components rejected
print(f' on average {df["0"].mean()} components were rejected')
print(f' the sdt of components rejected is {df["0"].std()}')
print(f' the maximum of components rejected is {df["0"].max()}')
print(f' the minimum of components rejected is {df["0"].min()}')

2. Evoked potentials

In [ ]:
# compare evokeds and plot contrasts
evokeds = evo.create_contrast(study=2, drop_bads=True, laplace=False,
                              subtract_evoked=False, save_data_to_disk=False,
                              save_drop_log=False)

In [ ]:
# plot evoked contrast and topography for the contrast
evo.plot_roi(study=2, data=evokeds, tmin=-0.1, tmax=0.3, tmin_base=-0.1, tmax_base=0)

3. Time-frequency analysis

In [ ]:
# compute tfr representations for each condition
tfr.compute_tfr(study=2, cond='prev_resp', tmin=-0.4, tmax=0, fmax=35,
 fmin=3, laplace=False, induced=False, mirror=True, drop_bads=True)

stimulus probability contrast

In [ ]:
# load the tfr data for each condition for probability conds.
tfr_a_cond, tfr_b_cond = tfr.load_tfr_conds(studies=[1, 2], cond='probability',
                                            cond_a_name='high_mirror',
                                            cond_b_name='low_mirror',
                                            cond_a_names=['high_prevhit_mirror',
                                                          'high_prevmiss_mirror',
                                                          'high_prevcr_mirror'],
                                            cond_b_names=['low_prevhit_mirror',
                                                          'low_prevmiss_mirror',
                                                          'low_prevcr_mirror'])

Qualitative checks for TFR (no stats yet)

In [ ]:
# plot grand average per condition (no differences, Gabriel Curios comments, numbtouch symposium)

# study 1
high_study1 = np.array(tfr_a_cond[0])
low_study1 = np.array(tfr_b_cond[0])

# study 2
high_study2 = np.array(tfr_a_cond[1])
low_study2 = np.array(tfr_b_cond[1])

# study 1: prevhits
prevhit_highstudy1 = high_study1[:,0]
prevhit_lowstudy1 = low_study1[:,0]

# grand average over participants
# study 1
prevhit_highstudy1gra = mne.grand_average([h for h in prevhit_highstudy1])
prevhit_lowstudy1gra = mne.grand_average([l for l in prevhit_lowstudy1])

high_study2gra = mne.grand_average([h for h in high_study2])
low_study2gra = mne.grand_average([l for l in low_study2])

# plot grand average
# study 1
diff = mne.combine_evoked([prevhit_highstudy1gra, prevhit_lowstudy1gra], weights=[1, -1])
diff.copy().crop(-0.4, 0).apply_baseline((-0.4,0), mode='zscore').plot(picks=['CP4'])

diff = mne.combine_evoked([high_study2gra, low_study2gra], weights=[1, -1])
diff.copy().crop(-0.4, 0).apply_baseline((-0.4,0), mode='zscore').plot(picks=['CP4'])

In [ ]:
study = 1 # expecon 2, single trial cues

# pick 10 random participants
random_ids = random.sample(range(0, len(tfr_a_cond[1])), 5)

# create figure with 3 rows and 5 columns
fig, axs = plt.subplots(3, 5, figsize=(15, 10))

# now fill the figure with the plots
for i, id in enumerate(random_ids):
    # plot tfr for each condition
    tfr_a_cond[study][id].copy().crop(-0.4, 0).plot(picks=['CP4'], axes=axs[0, i], show=False)
    tfr_b_cond[study][id].crop(-0.4, 0).plot(picks=['CP4'], axes=axs[1, i], show=False)

    diff = tfr_a_cond[study][id].copy().crop(-0.4, 0) - tfr_b_cond[study][id].crop(-0.4, 0)
    diff.plot(picks=['CP4'], axes=axs[2, i], show=False)
    # get rid of y label for every plot expcept the first one on the left
    axs[0, i].set_ylabel('')
    axs[1, i].set_ylabel('')
    axs[2, i].set_ylabel('')
    # also remove x axis for each row except the last row
    axs[0, i].set_xlabel('')
    axs[1, i].set_xlabel('')
    axs[2, i].set_xlabel('')
    # set title for each plot
    axs[0, i].set_title(f'ID {id}')

In [ ]:
study = 0 # mini block design, study 1
conds = [0, 1, 2]  # prev hit, prev miss, prev cr

# pick 10 random participants
random_ids = random.sample(range(0, len(tfr_a_cond[0])), 5)

for c in conds:
    # create figure with 3 rows and 5 columns
    fig, axs = plt.subplots(3, 5, figsize=(15, 10))
    # now fill the figure with the plots
    for i, id in enumerate(random_ids):
        # plot tfr for each condition
        tfr_a_cond[study][id][c].copy().crop(-0.4, 0).plot(picks=['CP4'], axes=axs[0, i], show=False)
        tfr_b_cond[study][id][c].crop(-0.4, 0).plot(picks=['CP4'], axes=axs[1, i], show=False)

        diff = tfr_a_cond[study][id][c].copy().crop(-0.4, 0) - tfr_b_cond[study][id][c].crop(-0.4, 0)
        diff.plot(picks=['CP4'], axes=axs[2, i], show=False)
        # get rid of y label for every plot expcept the first one on the left
        axs[0, i].set_ylabel('')
        axs[1, i].set_ylabel('')
        axs[2, i].set_ylabel('')
        # also remove x axis for each row except the last row
        axs[0, i].set_xlabel('')
        axs[1, i].set_xlabel('')
        axs[2, i].set_xlabel('')
        # set title for each plot
        axs[0, i].set_title(f'ID {id}')

In [ ]:
# run cluster based permutation tests for the conditions contrasts
# and plot sign. cluster
tfr.plot_tfr_cluster_test_output(cond='probability', tfr_a_cond=tfr_a_cond, tfr_b_cond=tfr_b_cond,
threed_test=False, cond_a_name='high', cond_b_name='low', channel_names=['CP4'])

previous response contrast

In [ ]:
# load the tfr data for each condition for prev_resp conds.
tfr_a_cond, tfr_b_cond = tfr.load_tfr_conds(studies=[1, 2], cond='prev_resp',
                                            cond_a_name='prevyesresp_highprob_stim_mirror',
                                            cond_b_name='prevnoresp_highprob_stim_mirror',
                                            cond_a_names=None, cond_b_names=None)

In [ ]:
# run cluster based permutation tests for the conditions contrasts
# and plot sign. cluster
tfr.plot_tfr_cluster_test_output(cond='prev_resp', tfr_a_cond=tfr_a_cond, tfr_b_cond=tfr_b_cond,
                                 threed_test=False, cond_a_name='prevyesresp',
                                 cond_b_name='prevnoresp', channel_names=['CP4'])

In [ ]:
# add alpha and beta power per trial to behavioral data and save in csv file for further analysis in R
tfr_trial.save_band_power_per_trial(study=2,
                                    time_intervals={'pre': [(-0.2,0), (-0.3, -0.1)]},
                                    channel_names=['CP4'],
                                    mirror=True)

4. Source reconstruction

In [ ]:
# run source reconstruction for each condition
source_reco.run_source_reco(study=2,
                            cond="probability",
                            mirror=True,
                            dics=True, fmin=15, fmax=25,
                            tmin=-0.4, tmax=0,
                            drop_bads=True,
                            plot_alignment=False)

In [ ]:
# plot source contrast (grand average over all participants)
# opens plots in separate windows
source_reco.plot_grand_average_source_contrast(study=2, cond='prev_resp', method='beamformer',
                                               save_plots=False, backend='matplotlib')

### Intermediate summary
Analysis of eeg data showed that in somatosensory regions, prestimulus beta power codes stimulus expectations in the volatile environment. For the stable environment prestimulus beta power codes the previous response. Source reconstruction shows the beta power source for stimulus probability in S2 and for previous responses in posterior cortex and ACC.